In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import sys
import importlib
from bs4 import BeautifulSoup

In [2]:
sys.path.append('..')

In [3]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('dynamo-covid-3c488865a4a3.json', scope)
client = gspread.authorize(creds)

In [4]:
#files=[x['name'] for x in client.list_spreadsheet_files()]

In [5]:
#files

In [6]:
#file_metadata = {
#    'name': 'DYNAMO-COVID-TAIL',
#    'mimeType': 'application/vnd.google-apps.spreadsheet',
#}
#client.create('DYNAMO-COVID-TAIL')

In [7]:
#data_id=client.open("DYNAMO-COVID").worksheets()[0].id
#countries_id=client.open("DYNAMO-COVID").worksheets()[1].id

In [8]:
# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
data = client.open("DYNAMO-COVID").get_worksheet(0)
countries=client.open("DYNAMO-COVID").get_worksheet(1)
tail=client.open("DYNAMO-COVID-TAIL").get_worksheet(0)

# Extract and print all of the values
#list_of_hashes = sheet.get_all_records()
#print(list_of_hashes)

In [9]:
#JOHNS HOPKINS

#url='https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports'

#html=requests.get(url).text
#soup = BeautifulSoup(html, 'html.parser')
#urls=[]
#for a in soup.find_all('a', href=True):
#    urls.append(a['href'])
#urls=['https://raw.githubusercontent.com'+x for x in urls if x[-4:]=='.csv']
#urls=[x.replace('/blob','') for x in urls]

#df=[]
#for link in urls:
#    print('processing file: '+link)
#    df.append(pd.read_csv(link,encoding='utf-8'))
#df=pd.concat(df)

In [10]:
#OUR WORLD IN DATA
url='https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-data.csv'
owid=pd.read_csv(url,encoding='utf-8')

In [11]:
#countries attributes reduced table
countries_cols=['iso_code', 'continent', 'location',
       'population', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index']
owid_countries=owid[countries_cols]
owid_countries=owid_countries.drop_duplicates(subset=['location'],keep='first')
owid_countries_list=owid_countries.fillna('').values.tolist()
owid_countries_list.insert(0,owid_countries.columns.to_list())

In [12]:
#remove countries data from main table
countries_cols.remove('iso_code')
owid=owid.drop(countries_cols,axis=1)
owid=owid.sort_values(by=['date','iso_code'],ascending=False)

In [13]:
#----TO BE IMPLEMENTED---SPLIT DATA AND UNION ON TABLEAU (for not exceeding sheet size 10 mb limit)
last_dates=owid['date'].unique()[:250]
owid_tail=owid[~owid['date'].isin(last_dates)]
owid=owid[owid['date'].isin(last_dates)]

In [14]:
owid_list=owid.fillna('').values.tolist()
owid_list.insert(0,owid.columns.to_list())

#----TO BE IMPLEMENTED---SPLIT DATA AND UNION ON TABLEAU (for not exceeding sheet size 10 mb limit)
owid_tail_list=owid_tail.fillna('').values.tolist()
owid_tail_list.insert(0,owid.columns.to_list())

In [15]:
data.update('A1', owid_list)

{'spreadsheetId': '1h7b4rgNIAwdf1lqjF2B3rb5v1TCPEHrkU9--hVkpuNQ',
 'updatedRange': 'DATA!A1:AP47033',
 'updatedRows': 47033,
 'updatedColumns': 42,
 'updatedCells': 1975386}

In [16]:
#----TO BE IMPLEMENTED---SPLIT DATA AND UNION ON TABLEAU
tail.update('A1', owid_tail_list)

{'spreadsheetId': '1NBzye4JNEzX4qPf_Cq66pQSsSN0uZl6ip_am_uoZEUk',
 'updatedRange': "'DATA-TAIL'!A1:AP16171",
 'updatedRows': 16171,
 'updatedColumns': 42,
 'updatedCells': 679182}

In [17]:
countries.update('A1', owid_countries_list)

{'spreadsheetId': '1h7b4rgNIAwdf1lqjF2B3rb5v1TCPEHrkU9--hVkpuNQ',
 'updatedRange': 'COUNTRIES!A1:R194',
 'updatedRows': 194,
 'updatedColumns': 18,
 'updatedCells': 3492}

In [18]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_WORLD.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/WORLD.ipynb')

0